# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

spark = SparkSession \
    .builder \
    .appName("DataFrames Practice") \
    .getOrCreate()

df = spark.read.json('data/sparkify_log_small.json')

# Question 1

Which page did user id "" (empty string) NOT visit?

In [8]:
# TODO: write your code to answer question 1
blank_pages = df.filter(df.userId == '').select('page').dropDuplicates()

In [9]:
all_pages = df.select('page').dropDuplicates()

In [10]:
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Save Settings
Submit Downgrade
Upgrade
Submit Upgrade
Downgrade
NextSong
Error
Logout
Settings


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [2]:
# TODO: use this space to explore the behavior of the user with an empty string


# Question 3

How many female users do we have in the data set?

In [16]:
# TODO: write your code to answer question 3
df.filter(df.gender == 'F').select('userID').dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [13]:
# TODO: write your code to answer question 4
df.filter(df.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [18]:
# TODO: write your code to answer question 5

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.head(20)

[Row(userID='1436', page='NextSong', ts=1513783259284, homevisit=0, period=0),
 Row(userID='1436', page='NextSong', ts=1513782858284, homevisit=0, period=0),
 Row(userID='2088', page='Home', ts=1513805972284, homevisit=1, period=1),
 Row(userID='2088', page='NextSong', ts=1513805859284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513805494284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513805065284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513804786284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513804555284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513804196284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513803967284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513803820284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513803651284, homevisit=0, period=1),
 Row(userID='2088', page='NextSong', ts=1513803413284, h